In [ ]:
from collections import namedtuple
import copy

Utilities = namedtuple("Utilities", ["tid", "pu", "nu", "ru"])

class Item:
    def __init__(self, item: str, utility: int):
        self.item = item
        self.utility = utility
        self._twu = 0

    @property
    def twu(self) -> int:
        return self._twu

    @twu.setter
    def twu(self, value: int) -> None:
        self._twu = value

    def __repr__(self):
        # return f"({self.item},{self.twu})"
        return f"{self.item}"

    def __eq__(self, other):
        if isinstance(other, Item):
            return self.item == other.item and self.utility == other.utility
        return False

    def __hash__(self):
        return hash((self.item, self.utility))


class Transaction:
    def __init__(self, id: int, items_quantities: dict):
        if any(q <= 0 for q in items_quantities.values()):
            raise ValueError(f"Quantities in trans{id} must be positive integers.")
        self.id = id
        self.items_quantities = items_quantities

    def __repr__(self):
        return f"(tid = {self.id}, frequencies = {self.items_quantities})"

class AbstractList:
    def __init__(self, items: set[Item], utility_values: list[Utilities]):
        self.items = items
        self.utility_values = utility_values

    
    def get_ru(self):
        ru = 0
        for i in self.utility_values:
            ru += i.ru
        return ru

    def get_pu(self):
        pu = 0
        for i in self.utility_values:
            pu += i.pu
        return pu


class PNUList(AbstractList):
    def __init__(self, items: set[Item], utility_values: list[Utilities]):
        super().__init__(items, utility_values)

    def __repr__(self):
        if not self.utility_values:
            return "Empty PNU-List"

        # Column titles
        titles = ["PU", "NU", "RU"]

        # Get the number of columns from the first utility value
        if isinstance(self.utility_values[0], (list, tuple)):
            num_columns = len(self.utility_values[0])
        else:
            num_columns = 1

        # Create combined items string
        items_str = ",".join(str(item) for item in self.items)
        items_str = "(" + items_str + ")"

        # Calculate column widths based on utility values and titles
        value_widths = []
        for i in range(num_columns):
            max_width = max(
                len(str(row[i])) if isinstance(row, (list, tuple)) else len(str(row))
                for row in self.utility_values
            )
            # Consider width of titles and combined items
            if i == 0:
                max_width = max(max_width, len(items_str))
            else:
                max_width = max(max_width, len(titles[i - 1]))
            value_widths.append(max_width)

        # Build the table string
        result = []

        # Add border
        total_width = sum(value_widths) + 3 * num_columns + 1
        result.append("-" * total_width)

        # Add single row with all items and titles
        row = "|"
        row += f" {items_str.rjust(value_widths[0])} |"
        for i in range(1, num_columns):
            row += f" {titles[i-1].center(value_widths[i])} |"
        result.append(row)

        # Add separator
        result.append("-" * total_width)

        # Add utility values
        for utility in self.utility_values:
            row = "|"
            if isinstance(utility, (list, tuple)):
                for i, value in enumerate(utility):
                    row += f" {str(value).rjust(value_widths[i])} |"
            else:
                row += f" {str(utility).rjust(value_widths[0])} |"
            result.append(row)

        # Add bottom border
        result.append("-" * total_width)
        return "\n".join(result)

# Subclass MList inheriting from AbstractList
class MList(AbstractList):
    def __init__(
        self,
        items: set[Item],
        true_items: set[Item],
        prefix: PNUList,
        utility_values: list[Utilities],
        ru: int,
        pu: int,
    ):
        super().__init__(items, utility_values)
        self.true_items = true_items
        self.prefix = prefix 
        self.ru = ru
        self.pu = pu

    def __repr__(self):
        return (f"MList(items={repr(self.items)}, "
                f"true_items={repr(self.true_items)}, "
                f"utility_values={repr(self.utility_values)}, "
                f"ru={self.ru}, pu={self.pu})")